![](img/Continuum_Logo_0702.png)

# Scientific Programming using Anaconda

## David Mertz

### dmertz@continuum.io

### 2016-04-04

This tutorial, and Python in general, run more smoothly under Python 3.x.

Whether you're running on Python 2 or Python 3, please install [Python-Future](http://python-future.org/futurize.html):
```bash
conda install future
```

In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from future import standard_library
standard_library.install_aliases()
from future.builtins import (
         bytes, dict, int, list, object, range, str,
         ascii, chr, hex, input, next, oct, open,
         pow, round, super, filter, map, zip)

# Table of Contents
* [Learning Objectives:](#Learning-Objectives:)
* [Not-quite-CSV: Eyeballing the data](#Not-quite-CSV:-Eyeballing-the-data)
* [Learning Objectives](#Learning-Objectives)
	* [Preamble](#Preamble)
* [Sqlite3](#Sqlite3)


# Learning Objectives:

After completion of this module, learners should be able to:

* Read from and write to delimited data files, such as CSV
* Learn how to do so robustly
* Learn why to not do so if possible
* Understand the structure of Excel .xlsx files
* Read data from Excel files
* Write data to Excel files
* Learn what JSON, YAML, and XML are
* Learn when and why to use them
* Learn how to manipulate and construct each type
* Learn the limitations and risks associated with each
* Work with formats that mirror the native data structures of Python:
* JSON
* YAML
* Work with XML data using several APIs:
* expat
* ElementTree
* SAX (Simple API for XML)
* DOM (Document Object Model
* Work with data stored in fast, hierarchical scientific data formats:
* HDF5
* NetCDF
* IDL .sav files
* Fortran 77 Unformatted

# Not-quite-CSV: Eyeballing the data

In [2]:
# More convoluted tab-separated with header lines, etc.
# Let's try to figure out how to work with the data
cowlitz_file = 'data/cowlitz_river_wa_usgs_flow_data.rdb'
cowlitz = open(cowlitz_file).readlines()
for line in cowlitz[:32]:
    print(line.rstrip())

# ---------------------------------- WARNING ----------------------------------------
# The data you have obtained from this automated U.S. Geological Survey database
# have not received Director's approval and as such are provisional and subject to
# revision.  The data are released on the condition that neither the USGS nor the
# United States Government may be held liable for any damages resulting from its use.
# Additional info: http://help.waterdata.usgs.gov/policies/provisional-data-statement
#
# File-format description:  http://help.waterdata.usgs.gov/faq/about-tab-delimited-output
# Automated-retrieval info: http://help.waterdata.usgs.gov/faq/automated-retrievals
#
# Contact:   gs-w_support_nwisweb@usgs.gov
# retrieved: 2015-06-11 16:58:34 EDT       (sdww01)
#
# Data for the following 1 site(s) are contained in this file
#    USGS 14226500 COWLITZ RIVER AT PACKWOOD, WA
# -----------------------------------------------------------------------------------
#
# Data provided for si

Subject area experts will find the format familiar, I am sure.  The *rdb* format is described at http://help.waterdata.usgs.gov/faq/about-tab-delimited-output as well.  But I am a non-expert in the subject area, so I will just visually examine it, and figure out in a relatively ad hoc way how to read and utilize it.

Here are some things I notice:

* The file starts with a commented header, with each line beginning with a hash mark (`# `) and space.
* The next line after the header is a list of field names.
 * Some field names start with numbers, and are not valid Python identifiers.
* The next line after the field names is the data types of the columns; but I'm not sure exactly what those descriptions mean.
* The bulk of the file is tab-separated values.

Let's write a small custom function to parse what we see in this data format. Note that I actually *did* a quick search, and it appears the modules `Asciitable` and the package `Astropy` both seem to support this format (other existing libraries might also); but suppose it was something novel.

In [3]:
def read_rdb(filename):
    import csv
    from collections import namedtuple, OrderedDict
    fh = open(filename)
    # First collect the comments, stopping at the field names
    comment_lines = []
    for line in fh:
        # We've gotten to the header
        if not line.startswith('#'):
            fields = line.rstrip().split('\t')
            break
        comment_lines.append(line[2:])
    # Make the individual lines into one string
    comment = ''.join(comment_lines)
    # Read the next line with the data formats
    formats = next(fh).rstrip().split('\t')
    # Make sure field names are valid Python identifiers
    field_names = [f if f[0].isalpha() else 'N_'+f for f in fields]
    # Define header as ordered mapping of field name to data type
    header = OrderedDict(zip(field_names, formats))
    row = namedtuple('Row', field_names)
    records = []
    for values in csv.reader(fh, delimiter='\t'):
        records.append(row(*values))
    # Close the file before we leave
    fh.close()
    return comment, header, records

In [4]:
comment, header, cowlitz_data = read_rdb(cowlitz_file)
for field, datatype in header.items():
    print("%s: %s" % (field, datatype))

agency_cd: 5s
site_no: 15s
datetime: 20d
N_01_00060_00003: 14n
N_01_00060_00003_cd: 10s


In [5]:
print(comment)

---------------------------------- WARNING ----------------------------------------
The data you have obtained from this automated U.S. Geological Survey database
have not received Director's approval and as such are provisional and subject to
revision.  The data are released on the condition that neither the USGS nor the
United States Government may be held liable for any damages resulting from its use.
Additional info: http://help.waterdata.usgs.gov/policies/provisional-data-statement
File-format description:  http://help.waterdata.usgs.gov/faq/about-tab-delimited-output
Automated-retrieval info: http://help.waterdata.usgs.gov/faq/automated-retrievals
Contact:   gs-w_support_nwisweb@usgs.gov
retrieved: 2015-06-11 16:58:34 EDT       (sdww01)
Data for the following 1 site(s) are contained in this file
   USGS 14226500 COWLITZ RIVER AT PACKWOOD, WA
-----------------------------------------------------------------------------------
Data provided for site 14226500
   DD parameter statisti

In [6]:
print("%d records, show first five" % len(cowlitz_data))

print('----------')
for record in cowlitz_data[:5]:
    print(record)
    
print('----------')
print("Work with a particular record in a straightforward way")
my_row = cowlitz_data[1000]
print(my_row.datetime, my_row.site_no, my_row.N_01_00060_00003)

11118 records, show first five
----------
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-01', N_01_00060_00003='577', N_01_00060_00003_cd='A')
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-02', N_01_00060_00003='568', N_01_00060_00003_cd='A')
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-03', N_01_00060_00003='557', N_01_00060_00003_cd='A')
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-04', N_01_00060_00003='544', N_01_00060_00003_cd='A')
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-05', N_01_00060_00003='536', N_01_00060_00003_cd='A')
----------
Work with a particular record in a straightforward way
1987-09-28 14226500 269


In [7]:
len([r for r in cowlitz_data if r.N_01_00060_00003_cd=='A:e'])

908

In [8]:
import pandas as pd
pd.DataFrame(cowlitz_data, columns=header.keys())

,agency_cd,site_no,datetime,N_01_00060_00003,N_01_00060_00003_cd
0,USGS,14226500,1985-01-01,577,A
1,USGS,14226500,1985-01-02,568,A
2,USGS,14226500,1985-01-03,557,A
3,USGS,14226500,1985-01-04,544,A
4,USGS,14226500,1985-01-05,536,A
5,USGS,14226500,1985-01-06,532,A
6,USGS,14226500,1985-01-07,534,A
7,USGS,14226500,1985-01-08,530,A
8,USGS,14226500,1985-01-09,518,A
9,USGS,14226500,1985-01-10,508,A


# Learning Objectives

* Work with SQLite3 single-file databases
* Work with RDBMS's using the DBAPI standard

## Preamble

In [9]:
# Load some data we'll use for later examples
import src.rdb as rdb
cowlitz_file = 'data/cowlitz_river_wa_usgs_flow_data.rdb'
comment, header, cowlitz_data = rdb.read_rdb(cowlitz_file)

# Notice the form of this data is a list of namedtuples
print("%d rows of data" % len(cowlitz_data), end='\n----------\n')
for row in cowlitz_data[:3]:
    print(row)

11118 rows of data
----------
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-01', N_01_00060_00003='577', N_01_00060_00003_cd='A')
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-02', N_01_00060_00003='568', N_01_00060_00003_cd='A')
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-03', N_01_00060_00003='557', N_01_00060_00003_cd='A')


In [10]:
# Load some data we'll use for later examples
# Note the form of this data is a Pandas DataFrame
import pandas as pd
aapl = pd.read_csv('data/AAPL.csv', index_col='Date')
print("%d rows of data" % len(aapl), end='\n----------\n')
aapl[:3]

8514 rows of data
----------


,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2014-09-16,99.80,101.26,98.89,100.86,66818200,100.86
2014-09-15,102.81,103.05,101.44,101.63,61216500,101.63
2014-09-12,101.21,102.19,101.08,101.66,62626100,101.66


# Sqlite3

In [11]:
import os
try:
    os.remove('tmp/test-db')
except OSError:
    print("File already not there")
#!rm tmp/test-db

In [12]:
import sqlite3
db = sqlite3.connect("tmp/test-db")
db.execute("create table stocks "
           "(symbol text, shares integer, price real, "
           " primary key (symbol))")
db.commit()

In [13]:
db.execute("insert into stocks values (?, ?, ?)", ('IBM', 50, 91.10))
db.execute("insert into stocks values (?, ?, ?)", ('AAPL', 100, 123.45))
db.commit()

In [14]:
for row in db.execute("select * from stocks"):
    print(row)

('IBM', 50, 91.1)
('AAPL', 100, 123.45)


In [15]:
stocks = [('GOOG', 75, 380.13), ('AA', 100, 14.20), ('AIG', 124, 0.99)]
db.executemany("insert into stocks values (?, ?, ?)", stocks)
db.commit()

In [16]:
list(db.execute("select * from stocks"))

[('IBM', 50, 91.1),
 ('AAPL', 100, 123.45),
 ('GOOG', 75, 380.13),
 ('AA', 100, 14.2),
 ('AIG', 124, 0.99)]

In [17]:
list(db.execute("select symbol, price from stocks where shares >= 100"))

[('AAPL', 123.45), ('AA', 14.2), ('AIG', 0.99)]

In [18]:
db.execute("insert into stocks values (?, ?, ?)", ('IBM', 100, 124.5))
db.commit()

IntegrityError: UNIQUE constraint failed: stocks.symbol

In [19]:
!dir tmp\test-db

/bin/sh: dir: command not found


In [20]:
db.execute("CREATE TABLE cowlitz "
           "(agency_cd TEXT, site_no INTEGER, date DATE, "
           " discharge REAL, status TEXT, PRIMARY KEY (date))")
for row in cowlitz_data:
    db.execute("INSERT INTO cowlitz VALUES (?, ?, ?, ?, ?)", row)
db.commit()

In [21]:
est = db.execute('SELECT COUNT(*) FROM cowlitz WHERE status="A:e"')
list(est)

[(908,)]

In [22]:
for d in db.execute('SELECT * FROM cowlitz WHERE '
                    'date >= "1988-01-01" AND date < "1988-01-10"'):
    print(d)

('USGS', 14226500, '1988-01-01', 381.0, 'A')
('USGS', 14226500, '1988-01-02', 363.0, 'A')
('USGS', 14226500, '1988-01-03', 347.0, 'A')
('USGS', 14226500, '1988-01-04', 355.0, 'A')
('USGS', 14226500, '1988-01-05', 355.0, 'A')
('USGS', 14226500, '1988-01-06', 351.0, 'A')
('USGS', 14226500, '1988-01-07', 347.0, 'A')
('USGS', 14226500, '1988-01-08', 343.0, 'A')
('USGS', 14226500, '1988-01-09', 349.0, 'A')


In [23]:
%ls -l tmp/test-db
%ls -l $cowlitz_file

-rw-r--r--@ 1 dmertz  staff  610304 Apr 24 22:10 tmp/test-db
-rwxr-xr-x@ 1 dmertz  staff  354101 Apr 16 00:38 data/cowlitz_river_wa_usgs_flow_data.rdb*


In [24]:
# Need Pandas column names to be valid SQL column names
aapl['Adj_Close'] = aapl['Adj Close']
del aapl['Adj Close']
aapl[:3]

,Open,High,Low,Close,Volume,Adj_Close
Date,,,,,,
2014-09-16,99.80,101.26,98.89,100.86,66818200,100.86
2014-09-15,102.81,103.05,101.44,101.63,61216500,101.63
2014-09-12,101.21,102.19,101.08,101.66,62626100,101.66


In [25]:
aapl.to_sql('AAPL', db)

In [26]:
for row in db.execute("SELECT * FROM AAPL LIMIT 10"):
    print(row)

('2014-09-16', 99.8, 101.26, 98.89, 100.86, 66818200, 100.86)
('2014-09-15', 102.81, 103.05, 101.44, 101.63, 61216500, 101.63)
('2014-09-12', 101.21, 102.19, 101.08, 101.66, 62626100, 101.66)
('2014-09-11', 100.41, 101.44, 99.62, 101.43, 62353100, 101.43)
('2014-09-10', 98.01, 101.11, 97.76, 101.0, 100741900, 101.0)
('2014-09-09', 99.08, 103.08, 96.14, 97.99, 189560600, 97.99)
('2014-09-08', 99.3, 99.31, 98.05, 98.36, 46277800, 98.36)
('2014-09-05', 98.8, 99.39, 98.31, 98.97, 58353200, 98.97)
('2014-09-04', 98.85, 100.09, 97.79, 98.12, 85594800, 98.12)
('2014-09-03', 103.1, 103.2, 98.58, 98.94, 125233100, 98.94)
